In [1]:
import pandas as pd
import torch

In [34]:
dataset = pd.read_csv(r"C:\Users\hahuy\OneDrive\Work\School\Thesis_TrendDetection\data\ecomm-reviews\reviews-ecommerce.csv")
dataset

,comment,label,rate,Unnamed: 3
0,Áo bao đẹp ạ!!,POS,5,NaN
1,Tuyệt vời !,POS,5,NaN
2,2day ao khong giong trong.,NEG,1,NaN
3,"Mùi thơm,bôi lên da mềm da.",POS,5,NaN
4,"Vải đẹp, dày dặn.",POS,5,NaN
...,...,...,...,...
31455,Không đáng tiền.,NEG,1,NaN
31456,Quần rất đẹp.,POS,5,NaN
31457,Hàng đẹp đúng giá tiền.,POS,5,NaN
31458,Chất vải khá ổn.,POS,4,NaN


In [41]:
# Perform preprocessing
import re

# TODO use a teen code list
rep = {"k": "không", "ko": "không", "đc": "được", "dc": "được", "a": "anh", "e": "em", "v": "vậy", "vs": "với"} # define desired replacements here

# use these three lines to do the replacement
rep_ = dict((r"\b{}\b".format(k), v) for k, v in rep.items())
pattern = re.compile("|".join(rep_.keys()), flags=re.I)

dataset["comment"] = dataset["comment"].apply(lambda text: pattern.sub(lambda m: rep[re.escape(m.group(0)).lower()], text))

dataset

,comment,label,rate,Unnamed: 3
0,Áo bao đẹp ạ!!,POS,5,NaN
1,Tuyệt vời !,POS,5,NaN
2,2day ao khong giong trong.,NEG,1,NaN
3,"Mùi thơm,bôi lên da mềm da.",POS,5,NaN
4,"Vải đẹp, dày dặn.",POS,5,NaN
...,...,...,...,...
31455,Không đáng tiền.,NEG,1,NaN
31456,Quần rất đẹp.,POS,5,NaN
31457,Hàng đẹp đúng giá tiền.,POS,5,NaN
31458,Chất vải khá ổn.,POS,4,NaN


In [44]:
# Split dataset into training and test sets
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(dataset["comment"].values, dataset["label"].values, test_size=0.15, random_state=42)
train_texts, valid_texts, train_labels, valid_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=232)

print(train_texts.shape, valid_texts.shape, test_labels.shape)
print(train_texts[:5], train_labels[:5])
print(valid_texts[:5], valid_labels[:5])
print(test_texts[:5], test_labels[:5])

(21392,) (5349,) (4719,)
['Sẽ ủng hộ thêm😍😍😍.' 'cảm ơn shop, Chất lượng sản phẩm tuyệt vời.'
 'Nói chung là với giá tiền bỏ ra thì chất lượng như thế là tạm ổn.'
 'Đóng gói sản phẩm rất đẹp và chắc chắn Shop phục vụ rất tốt.'
 'Túi zip bạc, có gói chống ẩm, thêm cái muỗng nữa, thèmmmmmmm...có ai ngờ đó là cám, là cám nghen.'
 'Dùng để sào thì được chứ chiên rán không được vì nó hoàn toàn không chống dính.'
 'To lắm nhag.' 'Màu không giống như hình.'
 'Thời gian giao hàng rất nhanh Đóng gói sản phẩm rất đẹp và chắc chắn Chất lượng sản phẩm tuyệt vời Mình cao 167cm mặc size S vừa nhưng tại eo mình hơi nhỏ nên có rơi rộng eo xíu hihi.'
 'Đồ xinh lắm với cả giao hàng nhanh nữa 😘.'
 'Sản phẩm không như mong đợi,kém...' 'Chất vải nóng , toàn chất poly.'
 'Còn lại mọi thứ y hình.' 'Giá lại siêu rẻ nữa ạ.'
 'Shop phục vụ rất tốt Rất đáng tiền Thời gian giao hàng rất nhanh Áo mặc form đẹp, cotton nhẹ và mát.'
 'Chất lượng giày tốt.'
 'Hình in ngoài sau hơi nhạt Shop phục vụ khá tốt Thời gian gi

In [45]:
from transformers import AutoTokenizer
from vncorenlp import VnCoreNLP

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", cache_dir="../phobert-base")
rdrsegmenter = VnCoreNLP(
    "./VnCoreNLP/VnCoreNLP-1.1.1.jar",
    annotators="wseg,pos",
    max_heap_size="-Xmx2g",
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [47]:
# PREPARE DATA BEFORE FEEDING TO MODEL

# Segmentize
train_segmented = [rdrsegmenter.tokenize(text) for text in train_texts]
valid_segmented = [rdrsegmenter.tokenize(text) for text in valid_texts]
test_segmented = [rdrsegmenter.tokenize(text) for text in test_texts]

# Flatten and merge back into strings
def flatten(t):
    return [item for sublist in t for item in sublist]
train_segmented = [" ".join(flatten(t)) for t in train_segmented]
valid_segmented = [" ".join(flatten(t)) for t in valid_segmented]
test_segmented = [" ".join(flatten(t)) for t in test_segmented]

print(train_segmented[:5], valid_segmented[:5], test_segmented[:5])

['Shop giao hàng nhanh như chớp luôn .', 'Đóng_gói sp cẩn_thận .', 'Đợi test thử vài ngày nữa xem sao !', 'Shop phục_vụ rất tốt . Rất đáng tiền .', 'Áo hơi mỏng nhưng mặc đẹp , dây của áo hoàn_thiện không được tốt cho lắm , mong shop khắc_phục .'] ['Chất_lượng sản_phẩm tuyệt_vời .', 'Đặt màu vàng thì ra màu gì vậy shop ? ? ?', 'Tôi không hề nhận được sản_phẩm này .', 'Áo mỏng hơn ảnh chụp .', 'Tuyệt .'] ['Máy chụp rất ok .', 'Chất_lượng sản_phẩm tuyệt_vời Đóng_gói sản_phẩm rất đẹp và chắc_chắn Đóng_gói sản_phẩm rất đẹp và chắc_chắn Rất đáng tiền Thời_gian giao hàng rất nhanh , yêu shop .', 'Chất_lượng tam duoc .', 'Hang rât tôt ủng_hộ dài_dài .', 'Vải cũng đẹp nữa .']


In [53]:
# Tokenize
train_encodings = tokenizer(train_segmented, truncation=True, padding="max_length", max_length=256)
valid_encodings = tokenizer(valid_segmented, truncation=True, padding="max_length", max_length=256)
test_encodings = tokenizer(test_segmented, truncation=True, padding="max_length", max_length=256)

Đóng_gói sp cẩn_thận .
[0, 55662, 1685, 36150, 3289, 5, 2, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 0, 0, 0]


In [55]:
print(train_segmented[2])
print(train_encodings["input_ids"][2][:20])
print(train_encodings["attention_mask"][2][:20])

Đợi test thử vài ngày nữa xem sao !
[0, 15621, 13923, 1176, 515, 43, 348, 305, 423, 381, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


[0, 55662, 1685, 36150, 3289, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [61]:
import torch

class EcommReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
        self.label_to_idx = {label: idx for idx, label in enumerate(set(labels))}
        self.idx_to_label = {idx: label for label, idx in self.label_to_idx.items()}

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.label_to_idx[self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = EcommReviewsDataset(train_encodings, train_labels)
val_dataset = EcommReviewsDataset(valid_encodings, valid_labels)
test_dataset = EcommReviewsDataset(test_encodings, test_labels)

In [63]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=3, cache_dir="../phobert-base")

training_args = TrainingArguments(
    output_dir='./sentiment-results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

loading configuration file https://huggingface.co/vinai/phobert-base/resolve/main/config.json from cache at ../phobert-base\a596f267f08b7158c7ab6300b1bf98eb6e1b05e6bcb0d7c18a8070364ee3011b.bbe27b2cac909b2279c83792c2d2b6f159f0a95f5d1c1eb66451da1c89a53609
Model config RobertaConfig {
  "_name_or_path": "vinai/phobert-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 258,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type

RuntimeError: CUDA out of memory. Tried to allocate 188.00 MiB (GPU 0; 4.00 GiB total capacity; 2.92 GiB already allocated; 0 bytes free; 2.99 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=len(dataset.label_to_ix), cache_dir="phobert-base")